In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
# df_train = df_train.iloc[:-686]

In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

36

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
# df_latest[df_latest['player'] == 'Chris Wood']
# lucas digne
# chris wood

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [11]:
last_play_proba.head()

,player,team,play_proba
0,Pontus Jansson,Brentford,1.0
1,Christian Nørgaard,Brentford,1.0
2,Ivan Toney,Brentford,1.0
3,Tariqe Fosu-Henry,Brentford,1.0
4,David Raya Martin,Brentford,1.0


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [13]:
df_latest.drop_duplicates(inplace=True)

In [14]:
df_train.drop_duplicates(inplace=True)

In [15]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [16]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [17]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [18]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
João Cancelo
Rodri
Bernardo Silva
Aymeric Laporte
Rúben Dias
Kevin De Bruyne
Raheem Sterling
Phil Foden
Jack Grealish
İlkay Gündoğan
Gabriel Jesus
Kyle Walker
Riyad Mahrez
John Stones
Oleksandr Zinchenko
Nathan Aké
Fernandinho
Ferrán Torres
Cole Palmer
Zack Steffen
Benjamin Mendy
James Mcatee
Liam Delap
Kayky Chagas
CJ Egan-Riley
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Alisson
Virgil van Dijk
Trent Alexander-Arnold
Sadio Mané
Mohamed Salah
Joël Matip
Jordan Henderson
Andrew Robertson
Fabinho
Diogo Jota
Thiago Alcántara
Naby Keïta
Roberto Firmino
Ibrahima Konaté
Luis Díaz
Curtis Jones
Kostas Tsimikas
James Milner
Alex Oxlade-Chamberlain
Harvey Elliott
Joe Gomez
Caoimhín Kelleher
Takumi Minamino
Tyler Morton
Divock Origi
Kaide Gordon
Neco Williams
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Edouard Mendy
Anton

Paul Dummett
Jeff Hendrick
Dwight Gayle
Crystal Palace
https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats
Marc Guéhi
Tyrick Mitchell
Conor Gallagher
Joachim Andersen
Wilfried Zaha
Vicente Guaita
Joel Ward
Jordan Ayew
Cheikhou Kouyaté
Jeffrey Schlupp
Odsonne Édouard
James McArthur
Nathaniel Clyne
Jean-Philippe Mateta
Michael Olise
Will Hughes
Christian Benteke
Luka Milivojević
Jack Butland
Eberechi Eze
James Tomkins
Jaïro Riedewald
Jesuran Rak Sakyi
Nathan Ferguson
Aston Villa
https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
Matty Cash
Emiliano Martínez
Tyrone Mings
John McGinn
Ollie Watkins
Douglas Luiz
Ezri Konsa
Jacob Ramsey
Danny Ings
Emi Buendía
Matt Targett
Philippe Coutinho
Lucas Digne
Ashley Young
Marvelous Nakamba
Calum Chambers
Leon Bailey
Axel Tuanzebe
Kortney Hause
Anwar El Ghazi
Morgan Sanson
Carney Chukwuemeka
Bertrand Traoré
Tim Iroegbunam
Jed Steer
Cameron Archer
Keinan Davis
Trézéguet
Jaden Philogene Bidace
Wesley Moraes
Southampton
https://fbref.com/en/squ

In [19]:
# df_enhanced_copy = df_enhanced.copy()

In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['round'].str[-2:] != 'on'].index]
df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['squad'] != 'CSKA Moscow'].index]

In [23]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [24]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [25]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10451,2021-2022,Jon Rowe,2022-04-23,Norwich,Premier League,34,1,1,19,11,...,0.1,1,0,11,13,84.6,14,1,1,Jonathan Rowe
10452,2021-2022,Jon Rowe,2022-04-30,Norwich,Premier League,35,0,0,9,4,...,0.0,0,0,3,4,75.0,7,0,3,Jonathan Rowe
10453,2021-2022,Jon Rowe,2022-05-11,Norwich,Premier League,21,0,0,7,14,...,0.0,0,0,2,3,66.7,4,0,0,Jonathan Rowe
10454,2021-2022,Jon Rowe,2022-05-15,Norwich,Premier League,37,0,0,5,0,...,0.1,1,0,3,5,60.0,5,1,1,Jonathan Rowe


In [26]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [33]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [28]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [29]:
# any additional corrections
match_list['CJ Egan-Riley'] = 'Conrad Egan-Riley'
# match_list['Josh Dasilva'] = 'Pelenda Joshua Dasilva'
# match_list['Bruno Guimarães'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Samir Santos'] = 'Samir Caetano de Souza Santos'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

# match_list['Sadio Man�'] = 'Sadio Mané'
# match_list['Jo�l Matip'] = 'Joel Matip'
# match_list['Thiago Alc�ntara'] = 'Thiago Alcántara do Nascimento'
# match_list['Naby Ke�ta'] = 'Naby Keita'
# match_list['Luis D�az'] = 'Luis Díaz'
# match_list['Jos� S�'] = 'José Malheiro de Sá'
# match_list['Romain Sa�ss'] = 'Romain Saïss'
# match_list['N�lson Semedo'] = 'Nélson Cabral Semedo'
# match_list['Ra�l Jim�nez'] = 'Raúl Jiménez'
# match_list['Jo�o Moutinho'] = 'João Filipe Iria Santos Moutinho'
# match_list['R�ben Neves'] = 'Rúben Diogo da Silva Neves'
# match_list['Rayan A�t Nouri'] = 'Rayan Ait Nouri'
# match_list['Fernando Mar�al'] = 'Fernando Marçal'
# match_list['Emiliano Mart�nez'] = 'Emiliano Martínez'
# match_list['Emi Buend�a'] = 'Emiliano Buendía Stati'
# match_list['Fabian Sch�r'] = 'Fabian Schär'
# match_list['Martin D�bravka'] = 'Martin Dubravka'
# match_list['Miguel Almir�n'] = 'Miguel Almirón'
# match_list['Bruno Guimar�es'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Kiko Femen�a'] = 'Francisco Femenía Far'

In [30]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [31]:
# make sure there are no non matches after update
# enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
# standard_teams = df_latest['team'].unique().tolist()

# unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
# unmatched_teams

[]

In [32]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [34]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [35]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [36]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [37]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [38]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [39]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [40]:
# add crowds field
df_latest_further['crowds'] = 1

In [41]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [42]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [43]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10451,2021-2022,Jon Rowe,2022-04-23,Norwich,Premier League,34,1,1,19,11,...,0.1,1,0,11,13,84.6,14,1,1,Jonathan Rowe
10452,2021-2022,Jon Rowe,2022-04-30,Norwich,Premier League,35,0,0,9,4,...,0.0,0,0,3,4,75.0,7,0,3,Jonathan Rowe
10453,2021-2022,Jon Rowe,2022-05-11,Norwich,Premier League,21,0,0,7,14,...,0.0,0,0,2,3,66.7,4,0,0,Jonathan Rowe
10454,2021-2022,Jon Rowe,2022-05-15,Norwich,Premier League,37,0,0,5,0,...,0.1,1,0,3,5,60.0,5,1,1,Jonathan Rowe


In [44]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [45]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
135348,Harry Kane,33,4,90,Tottenham Hotspur,Brighton and Hove Albion,NaN,NaN,True,2,...,0.3,2.0,0.0,12.0,18.0,66.7,18.0,1.0,1.0,1
136179,Harry Kane,34,4,90,Tottenham Hotspur,Brentford,NaN,NaN,False,2,...,0.0,1.0,0.0,10.0,15.0,66.7,17.0,0.0,2.0,1
136948,Harry Kane,35,4,90,Tottenham Hotspur,Leicester City,NaN,NaN,True,6,...,0.0,1.0,0.0,17.0,29.0,58.6,25.0,1.0,4.0,1
138421,Harry Kane,36,4,90,Tottenham Hotspur,Liverpool,NaN,NaN,False,2,...,0.1,5,1,9,20,45.0,19,3,3,1
138422,Harry Kane,36,4,90,Tottenham Hotspur,Arsenal,NaN,NaN,True,13,...,0.2,4,0,15,19,78.9,20,2,2,1


In [46]:
df_train_new.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
138619,N'Golo Kanté,36,3,0,Chelsea,Wolverhampton Wanderers,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138620,N'Golo Kanté,36,3,0,Chelsea,Leeds,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138621,Josh Martin,36,3,0,Norwich,West Ham United,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138622,Josh Martin,36,3,0,Norwich,Leicester City,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138623,Wilfred Ndidi,36,3,0,Leicester City,Everton,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138624,Wilfred Ndidi,36,3,0,Leicester City,Norwich,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138625,Matt Ritchie,36,2,0,Newcastle United,Manchester City,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138626,Nathan Redmond,36,3,90,Southampton,Brentford,NaN,NaN,False,2,...,0.1,5,0,33,45,73.3,43,1,1,1
138627,Mathew Ryan,36,1,0,Brighton and Hove Albion,Manchester United,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
138628,Ryan Fredericks,36,2,0,West Ham United,Norwich,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [47]:
df_latest_further.shape

(1198, 57)

In [48]:
df_train_new.shape

(138629, 59)

In [49]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [50]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [51]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
358,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,37,0
369,Chelsea,Leicester City,2022-05-19T19:00:00Z,37,1
367,Everton,Crystal Palace,2022-05-19T18:45:00Z,37,2
366,Southampton,Liverpool,2022-05-17T18:45:00Z,37,3
365,Newcastle United,Arsenal,2022-05-16T19:00:00Z,37,4
364,Everton,Brentford,2022-05-15T15:30:00Z,37,5
368,Aston Villa,Burnley,2022-05-19T19:00:00Z,37,6
362,West Ham United,Manchester City,2022-05-15T13:00:00Z,37,7
361,Watford,Leicester City,2022-05-15T13:00:00Z,37,8
360,Leeds,Brighton and Hove Albion,2022-05-15T13:00:00Z,37,9


In [52]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [53]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,6,90
1,37,1,Chelsea,Leicester City,2022-05-19T19:00:00Z,8,13
2,37,2,Everton,Crystal Palace,2022-05-19T18:45:00Z,11,31
3,37,3,Southampton,Liverpool,2022-05-17T18:45:00Z,20,14
4,37,4,Newcastle United,Arsenal,2022-05-16T19:00:00Z,4,3


In [54]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [55]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
3,Pierre-Emerick Aubameyang,0,4,4,96,3,Aubameyang
7,Pablo Marí,0,2,8,42,3,Marí
8,Héctor Bellerín,0,2,9,48,3,Bellerín
...,...,...,...,...,...,...,...
726,Christian Marques,0,2,549,40,39,Marques
728,Louie Moulden,0,1,611,40,39,Moulden
731,Bruno André Cavaco Jordão,0,3,661,45,39,Jordão
733,Dion Sanderson,0,2,686,40,39,Sanderson


In [56]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [57]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [58]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,37,0,Burnley,Tottenham Hotspur,2022-05-15T11:00:00Z,James Tarkowski,2,49,100,Tarkowski,False
1,37,0,Burnley,Tottenham Hotspur,2022-05-15T11:00:00Z,Phil Bardsley,2,39,100,Bardsley,False
2,37,0,Burnley,Tottenham Hotspur,2022-05-15T11:00:00Z,Erik Pieters,2,41,50,Pieters,False
3,37,0,Burnley,Tottenham Hotspur,2022-05-15T11:00:00Z,Jack Cork,3,44,100,Cork,False
4,37,0,Burnley,Tottenham Hotspur,2022-05-15T11:00:00Z,Dale Stephens,3,44,100,Stephens,False


In [59]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Ben Davies,2,45,100,Davies,True
1,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Joe Hart,1,45,0,Hart,True
2,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Hugo Lloris,1,55,None,Lloris,True
3,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Toby Alderweireld,2,50,0,Alderweireld,True
4,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Erik Lamela,3,60,0,Lamela,True


In [60]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [61]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [62]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Ben Davies,2,4.5,1.0,Davies,True,NaN,NaN,2122
1,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Joe Hart,1,4.5,0.0,Hart,True,NaN,NaN,2122
2,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Hugo Lloris,1,5.5,1.0,Lloris,True,NaN,NaN,2122
3,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Toby Alderweireld,2,5.0,0.0,Alderweireld,True,NaN,NaN,2122
4,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Erik Lamela,3,6.0,0.0,Lamela,True,NaN,NaN,2122
5,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Harry Kane,4,12.5,1.0,Kane,True,NaN,NaN,2122
6,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Serge Aurier,2,4.9,0.0,Aurier,True,NaN,NaN,2122
7,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Heung-Min Son,3,11.2,1.0,Son,True,NaN,NaN,2122
8,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Matt Doherty,2,4.5,0.0,Doherty,True,NaN,NaN,2122
9,37,0,Tottenham Hotspur,Burnley,2022-05-15T11:00:00Z,Eric Dier,2,4.7,1.0,Dier,True,NaN,NaN,2122


In [63]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')